# MACD in Python / Pandas

MACD(a, b, c) where:

a: is short moving average
b: is a longer moving average

Calculate the difference of a and b

Calculate the c-interval moving average

Common parameters are MACD(12,26,9)

For more background see: [MACD on Wikipedia](https://en.m.wikipedia.org/wiki/MACD)

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as dates

import matplotlib.dates as dates

# let's get some 1 minute bar data using alphavantage.co
# register for an APIKEY at alphavantage.co
# NOTE: you must set outputsize = full in your url otherwise you get truncated data
# SOURCE: https://www.alphavantage.co/

import io
import requests

# TODO: replace with your own API key
apikey = "G9X6PNM9DW17K47X"

# get daily stock prices
def getDailyStockPrices(symbol):
    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&symbol='+symbol+'&apikey='+apikey+'&datatype=csv'
    s = requests.get(url).content
    symbol_df = pd.read_csv(io.StringIO(s.decode('utf-8')))
    symbol_df = symbol_df.sort_values('timestamp', ascending=True)
    converted_timestamp = pd.to_datetime(symbol_df['timestamp'], infer_datetime_format=True)
    symbol_df.index = converted_timestamp
    symbol_df = symbol_df.drop(columns = ['timestamp'])
    return symbol_df

# get minute stock prices
def getMinuteStockPrices(symbol):
    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&interval=1min&outputsize=full&symbol='+symbol+'&apikey='+apikey+'&datatype=csv'
    s = requests.get(url).content
    symbol_df = pd.read_csv(io.StringIO(s.decode('utf-8')))
    symbol_df = symbol_df.sort_values('timestamp', ascending=True)
    converted_timestamp = pd.to_datetime(symbol_df['timestamp'], infer_datetime_format=True)
    symbol_df.index = converted_timestamp
    symbol_df = symbol_df.drop(['timestamp'], axis= 1)
    return symbol_df

# get minute stock prices
def getLatestStockPrice(symbol):
    url = 'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol=' + symbol + '&apikey=' + apikey + '&datatype=csv'
    s = requests.get(url).content
    symbol_df = pd.read_csv(io.StringIO(s.decode('utf-8')))
    return symbol_df['price'].values[0]


# set figsize...
plt.rcParams["figure.figsize"] = [16,16]


In [5]:
raw_data = getDailyStockPrices('TSLA')

In [6]:
stock_data = raw_data['2018-10-01':'2019-02-28']

In [7]:
# calculate EMAs

# 26 day
ema_26 = stock_data['close'].ewm(span=26).mean()

# 12 day
ema_12 = stock_data['close'].ewm(span=12).mean()

# difference between fast and slow averages
macd = ema_12 - ema_26

# calculate the 9 period EMA of the macd
macd_9_ema = macd.ewm(span=9).mean()

# calculate the difference the macd and the "signal line"
macd_diff = macd - macd_9_ema

In [8]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from mpl_finance import candlestick_ohlc

fig, (ax1, ax2) = plt.subplots(2,1, sharex=True)

# plot_day_summary(ax, quotes, ticksize=3)
candlestick_ohlc(ax1, zip(dates.date2num(stock_data.index.to_pydatetime()),
                         stock_data['open'], stock_data['high'],
                         stock_data['low'], stock_data['close']),
                         width=0.8, colorup='g', colordown='r')

#ax1.plot(stock_data['close'])
ax1.plot(ema_12)
ax1.plot(ema_26)
ax1.legend(['OHLC', 'EMA_12', 'EMA_26'])

ax2.plot(macd, "b-")
ax2.plot(macd_9_ema, "r-")
ax2.bar(macd.index, macd_diff)
ax2.legend(['macd', 'macd signal line', 'macd - signal line'])

ax2.xaxis.set_major_locator(dates.DayLocator(interval=5))
ax2.xaxis.set_major_formatter(dates.DateFormatter('%m-%d-%y'))

fig.tight_layout()
fig.autofmt_xdate()
fig.suptitle("MACD")

Text(0.5,0.98,'MACD')

## Now let's try to do make a trading strategy from this

In [ ]:
# given macd_diff, can we turn that into a series of signals?
macd_diff.head()

In [ ]:
def calc_signal(x):
    if x > 0:
        return 1 # buy
    elif x < 0:
        return -1 # sell
    else:
        return 0

In [ ]:
signal = macd_diff.apply(calc_signal)

In [ ]:
# now let's accumulate some P&L
position = 0
trade_size = 1 # 1 share
pnl = 0
previous_signal = 0
start_price = 0

for index, row in stock_data.iterrows():
    current_signal = signal.loc[index]
    if current_signal != previous_signal: # we need to do a trade
        # first check if we have an open position
        if position != 0: 
            # first we need to close the position and calculate P&L
            current_pnl = position * (row['open'] - start_price)
            pnl += current_pnl
            msg = "{0} Closing. signal: {1:d} current_price: {2:.2f} start_price: {3:.2f} pnl: {4:.2f}".format(index, current_signal, row['open'], start_price, current_pnl)
            print(msg)
            
        # now open a new position
        position = trade_size * current_signal # buy if signal is positive, sell if signal is negative
        start_price = row['open']
        msg = "{0} Opening. signal: {1:d} current_price: {2:.2f}".format(index, current_signal, row['open'])
        print(msg)    
        previous_signal = current_signal

print("Final P&L: {0:.2f}".format(pnl))